# Datenbanksysteme (SoSe22) Project
(names here)

## Project idea
(write)

## Datasets used
(write)

In [ ]:
# visualize datasets?

## ERM & relational model
(draw ERM, describe relational model)

## (explain process/challenges of setting up postgres database?)
(write)

## (explain data cleaning process)
(write + maybe some screenshots/show cleaned data?)

## (write about web app)